In [ ]:
import sys
import os 
import pandas as pd
import geopandas as gpd
from pathlib import Path
import json

In [ ]:
proj_dir = Path("./../../")

In [ ]:
sys.path.insert(0, os.path.abspath(proj_dir / "src"))

import thorr
from thorr.utils import config as cfg
from thorr.utils import database
from thorr.utils import logger

In [ ]:
config = cfg.read_config(config_path=str(proj_dir / ".env/config/thorr_config.ini"), required_sections=['mysql'])
db_config_path = config['mysql']["db_config_path"]

db = database.Connect(str(proj_dir / db_config_path))

In [ ]:
connection = db.connection
cursor = connection.cursor()

In [ ]:
db_name = "thorr_transfer_test"

In [ ]:
gpkg_path = proj_dir / "data/gis/geopackages/columbia_river_basin.gpkg"

In [ ]:
basins_gdf = gpd.read_file(gpkg_path, layer="Basins").to_crs(epsg=4326)
rivers_gdf = gpd.read_file(gpkg_path, layer="Rivers").to_crs(epsg=4326)
reaches_gdf = gpd.read_file(gpkg_path, layer="Reaches").to_crs(epsg=4326)
dams_gdf = gpd.read_file(gpkg_path, layer="Dams").to_crs(epsg=4326)
reservoirs_gdf = gpd.read_file(gpkg_path, layer="Reservoirs").to_crs(epsg=4326)

In [ ]:
basins_gdf.plot()

In [ ]:
# Insert basin data into the table if the entry doesn't already exist
for i, row in basins_gdf.iterrows():
    query = f"""
    INSERT INTO Basins (Name, DrainageAreaSqKm, geometry)
    SELECT '{row['Name']}', {row['AreaSqKm']}, ST_GeomFromText('{row['geometry'].wkt}', 4326, 'axis-order=long-lat')
    WHERE NOT EXISTS (SELECT * FROM Basins WHERE Name = '{row['Name']}')
    """
    
    cursor.execute(query)
    connection.commit()

In [ ]:
# Retrieve basin geometry from the database
cursor.execute("SELECT BasinID, Name, ST_AsText(geometry, 'axis-order=long-lat'), ST_SRID(geometry) AS SRID FROM basins")
basins = cursor.fetchall()
basins = pd.DataFrame(basins, columns=['basin_id', 'basin_name', 'geometry', 'srid'])

basins['geometry'] = gpd.GeoSeries.from_wkt(basins['geometry'])
basins = gpd.GeoDataFrame(basins, geometry='geometry')
basins = basins.set_crs(epsg=basins['srid'][0])
basins.plot()

In [ ]:
# Insert river data into the table if the entry doesn't already exist
for i, row in rivers_gdf.iterrows():
    query = f"""
    INSERT INTO Rivers (Name, LengthKm, geometry)
    SELECT '{row['GNIS_Name']}', {row['LengthKM']}, ST_GeomFromText('{row['geometry'].wkt}', 4326, 'axis-order=long-lat')
    WHERE NOT EXISTS (SELECT * FROM Rivers WHERE Name = '{row['GNIS_Name']}')
    """
    
    cursor.execute(query)
    connection.commit()

    # # Update the BasinID column if the basin exists in the Basins table
    # query2 = f"""
    # UPDATE Rivers
    # SET BasinID = (SELECT BasinID FROM Basins WHERE Name = '{row['Basin']}')
    # WHERE Name = '{row['GNIS_Name']}'
    # """
    query2 = f"""
    UPDATE Rivers
    SET BasinID = (SELECT BasinID FROM Basins WHERE Name = '{row['Basin']}'), LengthKm = {row['LengthKM']}
    WHERE Name = "{row['GNIS_Name']}"
    """

    cursor.execute(query2)
    connection.commit()

In [ ]:
# Update the MajorRiverID column if the river exists in the Rivers table
for i, row in basins_gdf.iterrows():
    query = f"""
    UPDATE Basins
    SET MajorRiverID = (SELECT RiverID FROM Rivers WHERE Name = '{row['MajorRiver']}')
    WHERE Name = '{row['Name']}'
    """

    cursor.execute(query)
    connection.commit()


In [ ]:
dams_gdf.fillna('', inplace=True)
reservoirs_gdf.fillna('', inplace=True)

# Insert river data into the table if the entry doesn't already exist
for i, row in dams_gdf.iterrows():
    # print(row['DAM_NAME'])
    query = f"""
    INSERT INTO Dams (Name, Reservoir, AltName, Country, Year, AreaSqKm, CapacityMCM, DepthM, ElevationMASL, MainUse, LONG_DD, LAT_DD, DamGeometry)
    SELECT "{row['DAM_NAME']}", NULLIF("{row['RES_NAME']}", ''), NULLIF("{str(row['ALT_NAME'])}",''), '{row['COUNTRY']}', {row['YEAR']}, {row['AREA_SKM']}, {row['CAP_MCM']}, {row['DEPTH_M']}, {row['ELEV_MASL']}, '{row['MAIN_USE']}', {row['LONG_DD']}, {row['LAT_DD']}, ST_PointFromText('{row['geometry'].wkt}', 4326, 'axis-order=long-lat')
    WHERE NOT EXISTS (SELECT * FROM Dams WHERE Name = "{row['DAM_NAME']}")
    """

    cursor.execute(query)
    connection.commit()


    # Update the RiverID column if the river exists in the Rivers table
    query2 = f"""
    UPDATE Dams
    SET RiverID = (SELECT RiverID FROM Rivers WHERE Name = "{row['RIVER']}")
    WHERE Name = "{row['DAM_NAME']}"
    """

    cursor.execute(query2)
    connection.commit()

    # Update the BasinID column if the basin exists in the Basins table
    query3 = f"""
    UPDATE Dams
    SET BasinID = (SELECT BasinID FROM Basins WHERE Name = 'Columbia River Basin')
    WHERE Name = "{row['DAM_NAME']}"
    """

    cursor.execute(query3)
    connection.commit()

# Insert reservoir data into the table if the entry doesn't already exist
for i, row in reservoirs_gdf.iterrows():
    query = f"""
    UPDATE Dams
    SET ReservoirGeometry = ST_GeomFromText('{row['geometry'].wkt}', 4326, 'axis-order=long-lat')
    WHERE Name = "{row['DAM_NAME']}"
    """

    cursor.execute(query)
    connection.commit()


In [ ]:
# Iinsert reach data into the table if the entry doesn't already exist
for i, row in reaches_gdf.iterrows():
    # query = f"""
    # INSERT INTO Reaches (Name, RiverID, ClimateClass, Width, Width5, Width95, Depth, Depth5, Depth95, geometry)
    # SELECT "{row['reach_id']}",(SELECT RiverID FROM Rivers WHERE Name = '{row['GNIS_Name']}'), {row['koppen']}, {row["WIDTH"]}, {row["WIDTH5"]}, {row["WIDTH95"]}, {row["DEPTH"]}, {row["DEPTH5"]}, {row["DEPTH95"]}, ST_GeomFromText('{row['geometry'].wkt}', 4326, 'axis-order=long-lat')
    # WHERE NOT EXISTS (SELECT * FROM Reaches WHERE Name = "{row['reach_id']}")
    # """

    query = f"""
    INSERT INTO Reaches (Name, RiverID, ClimateClass, WidthMin, WidthMean, WidthMax, RKm, geometry)
    SELECT "{row['reach_id']}",(SELECT RiverID FROM Rivers WHERE Name = '{row['GNIS_Name']}'), {row['koppen']}, NULLIF("{str(row['WidthMin'])}",'nan'), NULLIF("{str(row['WidthMean'])}",'nan'), NULLIF("{str(row['WidthMax'])}",'nan'), NULLIF("{str(row['RKm'])}",'nan'), ST_GeomFromText('{row['geometry'].wkt}', 4326, 'axis-order=long-lat')
    WHERE NOT EXISTS (SELECT * FROM Reaches WHERE Name = "{row['reach_id']}")
    """
    
    try:
        cursor.execute(query)
        connection.commit()
    except:
        print(query)
        break